In [ ]:
from datetime import date
import random
import time
import yfinance as yf
import pandas as pd

import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from numpy.fft import fft, ifft, fftshift
import numpy as np
from numpy import log, sqrt, exp


from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t, poisson
from scipy.linalg import solve_banded
from scipy.optimize import minimize, differential_evolution
from scipy.integrate import quad
from scipy.special import roots_laguerre
from scipy.interpolate import interp1d
from scipy.sparse import diags, kron, identity, csr_matrix
from scipy.sparse.linalg import spsolve

import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

#import pymc as pm
#import arviz as az

from tensorflow import keras
#from tensorflow.keras.utils import plot_model

#import pyswarms as ps

######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class bsm_options_value:

    def __init__(self, S0, K, T, r, sigma, option_type):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.option_type = option_type

    def analytical(self):
        d1 = (np.log(self.S0 / self.K) + (self.r + 0.5 * self.sigma ** 2) * self.T) / (self.sigma * np.sqrt(self.T))
        d2 = d1 - self.sigma * np.sqrt(self.T)

        if self.option_type == 'call':
            price = self.S0 * norm.cdf(d1) - self.K * np.exp(-self.r * self.T) * norm.cdf(d2)

            delta = norm.cdf(d1)
            gamma = norm.pdf(d1) / (self.S0 * self.sigma * np.sqrt(self.T))
            vega = (self.S0 * norm.pdf(d1) * np.sqrt(self.T)) / 100  # per 1% change in vol
            theta = (1/252) * (-self.S0 * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
                              - self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(d2))  # per business day
            rho = (self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(d2)) / 100

        elif self.option_type == 'put':
            price = self.K * np.exp(-self.r * self.T) * norm.cdf(-d2) - self.S0 * norm.cdf(-d1)

            delta = norm.cdf(d1) - 1
            gamma = norm.pdf(d1) / (self.S0 * self.sigma * np.sqrt(self.T))
            vega = (self.S0 * norm.pdf(d1) * np.sqrt(self.T)) / 100  # per 1% change in vol
            theta = (1/252) * (-self.S0 * norm.pdf(d1) * self.sigma / (2 * np.sqrt(self.T))
                              + self.r * self.K * np.exp(-self.r * self.T) * norm.cdf(-d2))  # per business day
            rho = (-self.K * self.T * np.exp(-self.r * self.T) * norm.cdf(-d2)) / 100

        return price , delta, gamma, theta, vega, rho

    def crank_nicolson_fd(self, exercise_type, exercise_days):
        max_iter = 100
        tol = 1e-6
        omega = 1.9

        M = 100
        N = 10000
        S_max = 2 * self.S0
        dS = S_max / M
        dt = self.T / N
        S = np.linspace(0, S_max, M + 1)

        if exercise_type == 'european':
            exercise_binary = np.zeros(N + 1)
        elif exercise_type == 'american':
            exercise_binary = np.ones(N + 1)
        elif exercise_type == 'bermudan':
            exercise_binary = np.zeros(N + 1)
            for i in range(N+1):
                if i in exercise_days:
                    exercise_binary[i] = 1

        # Terminal payoff
        if self.option_type == 'call':
            V = np.maximum(S - self.K, 0)
        elif self.option_type == 'put':
            V = np.maximum(self.K - S, 0)

        # Prepare tridiagonal matrix coefficients
        i = np.arange(1, M)
        a = -0.5 * dt * (self.sigma**2 * i**2 - self.r * i)
        b = 1 + dt * (self.sigma**2 * i**2 + self.r)
        c = -0.5 * dt * (self.sigma**2 * i**2 + self.r * i)

        A_lower = a.copy()
        A_diag  = b.copy()
        A_upper = c.copy()

        # Matrix B (explicit part)
        i = np.arange(1, M)
        alpha = 0.5 * dt * (self.sigma**2 * i**2 - self.r * i)
        beta  = 1 - dt * (self.sigma**2 * i**2 + self.r)
        gamma = 0.5 * dt * (self.sigma**2 * i**2 + self.r * i)

        B_lower = alpha.copy()
        B_diag = beta.copy()
        B_upper = gamma.copy()

        for n in range(N):
            t = self.T - n * dt

            rhs = B_lower * V[:-2] + B_diag * V[1:-1] + B_upper * V[2:]

            # Boundary conditions
            if self.option_type == 'call':
                V[0] = 0
                V[-1] = S_max - self.K * np.exp(-self.r * t)
            else:
                V[0] = self.K * np.exp(-self.r * t)
                V[-1] = 0

            rhs[0]  += a[0] * V[0]
            rhs[-1] += c[-1] * V[-1]

            # Solve A x = rhs
            if exercise_binary[n] == 1:
                # PSOR for American option
                x = V[1:M].copy()
                phi = np.maximum(S[1:M] - self.K, 0) if self.option_type == 'call' else np.maximum(self.K - S[1:M], 0)

                for k in range(max_iter):
                    x_old = x.copy()
                    for j in range(M - 1):
                        left  = A_lower[j] * (x[j - 1] if j > 0 else 0)
                        right = A_upper[j] * (x[j + 1] if j < M - 2 else 0)
                        x_new = (1 - omega) * x[j] + omega / A_diag[j] * (rhs[j] - left - right)
                        x[j] = max(x_new, phi[j])  # projection

                    if np.linalg.norm(x - x_old, np.inf) < tol:
                        break
                else:
                    print("Warning: PSOR did not converge")
            else:
                '''
                # Standard backward implicit solver for European option
                A = np.zeros((M - 1, M - 1))
                np.fill_diagonal(A, A_diag)
                np.fill_diagonal(A[1:], A_lower[1:])
                np.fill_diagonal(A[:, 1:], A_upper[:-1])
                x = np.linalg.solve(A, rhs)
                '''
                # Prepare banded matrix for scipy.linalg.solve_banded
                A = np.zeros((3, M-1))
                A[0, 1:]  = A_upper[:-1]      # Upper diagonal
                A[1, :]   = A_diag            # Main diagonal
                A[2, :-1] = A_lower[1:]       # Lower diagonal

                # Solve the linear system A x = rhs using banded solver
                x = solve_banded((1, 1), A, rhs)

            V[1:M] = x

        # Interpolation to get price at S0
        option_price_at_S0 = np.interp(self.S0, S, V)
        return option_price_at_S0


    def compute_greeks_fd(self, S, V, method):
        dS = S[1] - S[0]

        # Find closest index to S0
        idx = np.searchsorted(S, self.S0)
        if idx < 1 or idx >= len(S) - 1:
            print("S0 is too close to the boundary for finite difference.")
            return

        # First Derivative: Delta
        delta = (V[idx + 1] - V[idx - 1]) / (2 * dS)

        # Second Derivative: Gamma
        gamma = (V[idx + 1] - 2 * V[idx] + V[idx - 1]) / (dS ** 2)

        # Theta: Approximate using terminal payoff and one time step back
        dt = self.T / 1000
        S_val = S[idx]
        if self.option_type == 'call':
            payoff = max(S_val - self.K, 0)
        else:
            payoff = max(self.K - S_val, 0)
        theta = -(V[idx] - payoff) / dt  # Finite diff in time direction

        # Vega: Bump sigma and reprice using the same method
        eps = 1e-4
        orig_sigma = self.sigma
        self.sigma += eps
        if method == 'explicit_fd':
            V_eps, _, _, _, _ = self.explicit_fd()
        elif method == 'implicit_fd':
            V_eps, _, _, _, _  = self.implicit_fd()
        elif method == 'crank_nicolson_fd':
            V_eps, _, _, _, _  = self.crank_nicolson_fd()
        else:
            raise ValueError("Invalid method for computing Vega")
        self.sigma = orig_sigma
        vega = (V_eps - V[idx]) / eps

        # Rho: Bump interest rate and reprice
        orig_r = self.r
        self.r += eps
        if method == 'explicit_fd':
            V_eps_r, _, _, _, _ = self.explicit_fd()
        elif method == 'implicit_fd':
            V_eps_r, _, _, _, _ = self.implicit_fd()
        elif method == 'crank_nicolson_fd':
            V_eps_r, _, _, _, _ = self.crank_nicolson_fd()

        self.r = orig_r
        rho = (V_eps_r - V[idx]) / eps

        return delta, gamma, theta, vega, rho